# Goodreads Reading List Recommender Project (Data Cleaning)

In [1]:
# Import basic data libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import a function that will create a connection to MySQL
from sqlalchemy import create_engine

## Importing and Inspecting

In [2]:
# Load the reviews data into pd.DataFrame objects using pandas and set the date columns to date dtypes 
reviews = pd.read_csv(
    '../Data/goodreads_reviews.csv',
    parse_dates=['started_at', 'read_at', 'date_added']
)

# Load the works data into pd.DataFrame
books = pd.read_csv(
    '../Data/goodreads_works.csv',
    dtype={'isbn': str, 'isbn13': str}
)

In [3]:
# Make sure the reviews table loaded in correctly by checking the head
reviews.head(3)

,review_id,user_id,work_id,started_at,read_at,date_added,rating,review_text,n_votes,n_comments
0,fa7a00c01296e3b2b2e857d79c51ea77,3693bb4f1062b659a354848cf11ca313,6128277,NaN,NaN,2013-12-21,5.0,Fire is half-human and half monster. Monsters ...,0,0
1,de0f7c8d15e247443e51969becf2878e,3693bb4f1062b659a354848cf11ca313,3270810,NaN,NaN,2013-12-21,5.0,Katsa is a graceling - blessed with an ability...,0,0
2,e79b49504ef58b2defcdc8b79e2ec3fb,3693bb4f1062b659a354848cf11ca313,4768235,NaN,NaN,2013-12-19,5.0,"This is a fun, light-hearted read. Tammy Jo is...",0,0


In [4]:
# Make sure the books table loaded in correctly by checking the head
books.head(3)

,work_id,isbn,isbn13,original_title,author,original_publication_year,num_pages,description,genres,image_url,reviews_count,text_reviews_count,5_star_ratings,4_star_ratings,3_star_ratings,2_star_ratings,1_star_ratings,ratings_count,avg_rating,similar_books
0,2919130,1416534601,9781416534600,Nocturnes,John Connolly,2004.0,NaN,NaN,"fiction, fantasy, paranormal, mystery, thrille...",https://s.gr-assets.com/assets/nophoto/book/11...,8820,338,1118,1601,1029,190,58,3996,3.9,NaN
1,52087333,NaN,NaN,Draw Play,Tia Lewis,2016.0,NaN,Jake:\nI can't believe my coach assigned me a ...,"romance, fiction",https://s.gr-assets.com/assets/nophoto/book/11...,2482,204,204,353,274,77,29,937,3.7,NaN
2,1649583,1416505520,9781416505525,Citizen of the Galaxy,Robert A. Heinlein,1957.0,NaN,"In a distant galaxy, the atrocity of slavery w...","fiction, young-adult, fantasy, paranormal, chi...",https://s.gr-assets.com/assets/nophoto/book/11...,16506,447,3539,4351,2863,444,53,11250,4.0,NaN


## Cleaning and Wrangling

For each table, investigate and clean the following: 
- Data types will need to be converted
- Bit-sizes can be decreased
- Possible outliers may exist in numeric columns
- Missing data may exist
- Duplicates may exists
- Inconsistent text and typos may exist in categorical columns 

### Reviews Table

In [5]:
# Inspect the data types of the reviews table
reviews.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143887 entries, 0 to 1143886
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   review_id    1143887 non-null  object        
 1   user_id      1143887 non-null  object        
 2   work_id      1143887 non-null  int64         
 3   started_at   796392 non-null   object        
 4   read_at      1031765 non-null  object        
 5   date_added   1143887 non-null  datetime64[ns]
 6   rating       1106569 non-null  float64       
 7   review_text  1143887 non-null  object        
 8   n_votes      1143887 non-null  int64         
 9   n_comments   1143887 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 1.5 GB


The `started_at` and `read_at` columns should be datetime64 dtypes. All other data types look correct.

In [6]:
# Convert the started_at and read_at columns to datetime64 dtypes and turn all values that can not be parsed into NaT values
reviews['started_at'] = pd.to_datetime(reviews['started_at'], errors='coerce')
reviews['read_at'] = pd.to_datetime(reviews['read_at'], errors='coerce')

In [7]:
# Check to see if the changes occured correctly
reviews.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143887 entries, 0 to 1143886
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   review_id    1143887 non-null  object        
 1   user_id      1143887 non-null  object        
 2   work_id      1143887 non-null  int64         
 3   started_at   796391 non-null   datetime64[ns]
 4   read_at      1031763 non-null  datetime64[ns]
 5   date_added   1143887 non-null  datetime64[ns]
 6   rating       1106569 non-null  float64       
 7   review_text  1143887 non-null  object        
 8   n_votes      1143887 non-null  int64         
 9   n_comments   1143887 non-null  int64         
dtypes: datetime64[ns](3), float64(1), int64(3), object(3)
memory usage: 1.4 GB


We lost one value of the `started_at` column and two values of the `read_at` column using errors='coerce' in pd.to_datetime.

In [8]:
# Check the summary statistics for the numeric columns to see if bit-sizes can be reduced and identify possible outliers
reviews.describe().round()

,work_id,started_at,read_at,date_added,rating,n_votes,n_comments
count,1143887.0,796391,1031763,1143887,1106569.0,1143887.0,1143887.0
mean,17830071.0,2014-10-25 17:59:29.161881344,2014-05-27 04:23:58.279140096,2014-01-25 03:31:19.395080192,4.0,3.0,1.0
min,104.0,1916-01-01 00:00:00,1915-05-26 00:00:00,2006-08-29 00:00:00,1.0,-3.0,-1.0
25%,3443248.0,2013-05-27 00:00:00,2013-01-25 00:00:00,2012-07-31 00:00:00,3.0,0.0,0.0
50%,15956441.0,2015-01-24 00:00:00,2014-10-14 00:00:00,2014-03-29 00:00:00,4.0,0.0,0.0
75%,25032500.0,2016-05-31 00:00:00,2016-04-01 00:00:00,2015-10-22 00:00:00,5.0,1.0,0.0
max,57717521.0,2017-12-29 00:00:00,2017-12-31 00:00:00,2017-10-29 00:00:00,5.0,3222.0,833.0
std,14795716.0,NaN,NaN,NaN,1.0,18.0,5.0


* The `rating`, `n_votes`, and `n_comments` columns take values between -3 and 3222. I will reduce the bit-sizes for those columns to the minimum bit-size to reduce memory usage and optimise efficiency.
* Looking at the Goodreads website, you can not dislike a review. Therefore, the `n_votes` and `n_comments` columns likely have outliers since there are negative values. I will turn the negative values in these columns to 0.

In [9]:
# Turn the negative values in n_votes and n_comments to 0
reviews['n_votes'] = np.where(reviews['n_votes'] < 0, 0, reviews['n_votes'])
reviews['n_comments'] = np.where(reviews['n_comments'] < 0, 0, reviews['n_comments'])

In [10]:
# Check to see if the changes worked correctly
reviews.describe().round().loc['min',['n_votes', 'n_comments']]

n_votes       0.0
n_comments    0.0
Name: min, dtype: object

In [11]:
# Reduce the bit-sizes of the above stated columns
reviews = reviews.astype({
    'rating': 'float32',
    'n_votes': 'int16',
    'n_comments': 'int16'
})

In [12]:
# Check to see if the changes worked correctly
reviews.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143887 entries, 0 to 1143886
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   review_id    1143887 non-null  object        
 1   user_id      1143887 non-null  object        
 2   work_id      1143887 non-null  int64         
 3   started_at   796391 non-null   datetime64[ns]
 4   read_at      1031763 non-null  datetime64[ns]
 5   date_added   1143887 non-null  datetime64[ns]
 6   rating       1106569 non-null  float32       
 7   review_text  1143887 non-null  object        
 8   n_votes      1143887 non-null  int16         
 9   n_comments   1143887 non-null  int16         
dtypes: datetime64[ns](3), float32(1), int16(2), int64(1), object(3)
memory usage: 1.4 GB


In [13]:
# Check the value_counts of ratings to see if there are any anomolies
reviews['rating'].value_counts()

rating
4.0    396722
5.0    336246
3.0    242109
2.0     94030
1.0     37462
Name: count, dtype: int64

No anomolies in the `rating` column.

In [14]:
# Check the percentage of missing data for each column
(reviews.isna().sum() / reviews.shape[0]).round(2)

review_id      0.00
user_id        0.00
work_id        0.00
started_at     0.30
read_at        0.10
date_added     0.00
rating         0.03
review_text    0.00
n_votes        0.00
n_comments     0.00
dtype: float64

**NOTE TO SELF:**
* We can fill in the rating missing values with the sentiment of the review_text.
* If we care about how long it takes to read, is there any point in having a read_at value if we do not know the started_at value

**ALSO:**
* There are no text categorical columns

In [15]:
# Check for duplicates in the whole dataset
reviews.duplicated().sum()

np.int64(0)

In [16]:
# Check for duplicates in the review_id column
reviews.duplicated(subset=['review_id']).sum()

np.int64(0)

In [17]:
# Check if the same user reviewed the same book more than once
reviews.duplicated(subset=['user_id', 'work_id']).sum()

np.int64(2323)

In [18]:
# Inspect the rows where a user reviewed the same book more than once
reviews[reviews.duplicated(subset=['user_id', 'work_id'], keep=False)].sort_values(by='user_id').head(2)

,review_id,user_id,work_id,started_at,read_at,date_added,rating,review_text,n_votes,n_comments
529654,5173fae9e68df8cea6200ed7c7b1d875,00608f34a76da8ba3ed37b25f263adf4,12803237,2013-07-25,2013-09-14,2013-07-22,4.0,This was my first book from Laura Buzo and sho...,0,0
669062,2b05746c58293271e3cd76b9d66892df,00608f34a76da8ba3ed37b25f263adf4,12803237,2013-09-05,2013-09-14,2013-09-05,4.0,This was my first book from Laura Buzo and sho...,1,0


**NOTE TO SELF:**

* Export to SQL to further investigate the duplicates

### Books Table

In [19]:
# Inspect the data types of the books table
books.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13525 entries, 0 to 13524
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   work_id                    13525 non-null  int64  
 1   isbn                       11474 non-null  object 
 2   isbn13                     11864 non-null  object 
 3   original_title             13525 non-null  object 
 4   author                     13525 non-null  object 
 5   original_publication_year  13507 non-null  float64
 6   num_pages                  12795 non-null  float64
 7   description                13356 non-null  object 
 8   genres                     13525 non-null  object 
 9   image_url                  13525 non-null  object 
 10  reviews_count              13525 non-null  int64  
 11  text_reviews_count         13525 non-null  int64  
 12  5_star_ratings             13525 non-null  int64  
 13  4_star_ratings             13525 non-null  int

All data types look correct.

In [20]:
# Check the summary statistics for the numeric columns to see if bit-sizes can be reduced and identify possible outliers
books.describe().round()

,work_id,original_publication_year,num_pages,reviews_count,text_reviews_count,5_star_ratings,4_star_ratings,3_star_ratings,2_star_ratings,1_star_ratings,ratings_count,avg_rating
count,13525.0,13507.0,12795.0,13525.0,13525.0,13525.0,13525.0,13525.0,13525.0,13525.0,13525.0,13525.0
mean,18436259.0,2003.0,335.0,61441.0,2037.0,14398.0,12172.0,6903.0,1914.0,833.0,36220.0,4.0
std,16258712.0,59.0,138.0,203063.0,5429.0,70546.0,45515.0,25036.0,8474.0,5781.0,148374.0,0.0
min,104.0,-500.0,1.0,107.0,18.0,5.0,4.0,4.0,0.0,0.0,36.0,2.0
25%,2953328.0,2006.0,260.0,6804.0,286.0,759.0,947.0,596.0,146.0,44.0,2673.0,4.0
50%,15992791.0,2011.0,334.0,16588.0,664.0,2343.0,2650.0,1576.0,383.0,123.0,7351.0,4.0
75%,25878877.0,2014.0,391.0,44017.0,1709.0,7925.0,7854.0,4463.0,1100.0,369.0,22025.0,4.0
max,57717521.0,2021.0,2201.0,6057595.0,156575.0,3131920.0,1519190.0,808753.0,444888.0,463808.0,5066596.0,5.0


* `original_publication_year` could have an outlier since it has a value of -500.
* `num_pages` may have outliers since there is a book with a single page.

In [21]:
# Investigate the book with a release year of -500
books[books['original_publication_year'] == -500]

,work_id,isbn,isbn13,original_title,author,original_publication_year,num_pages,description,genres,image_url,reviews_count,text_reviews_count,5_star_ratings,4_star_ratings,3_star_ratings,2_star_ratings,1_star_ratings,ratings_count,avg_rating,similar_books
10917,3200649,1590302257,9781590302255,Sun Zi Bing Fa [Sunzi bingfa],Sun Tzu,-500.0,273.0,"Conflict is an inevitable part of life, accord...","non-fiction, history, historical fiction, biog...",https://images.gr-assets.com/books/1453417993m...,469134,6631,73919,73612,50069,12038,3074,212712,4.0,NaN


After research, this book was released 500 BC, meaning -500 in the `original_publication_year` column is not an anomoly.

In [22]:
# Investigate the books with the most reviews
books.sort_values('num_pages', ascending=True).head(3)

,work_id,isbn,isbn13,original_title,author,original_publication_year,num_pages,description,genres,image_url,reviews_count,text_reviews_count,5_star_ratings,4_star_ratings,3_star_ratings,2_star_ratings,1_star_ratings,ratings_count,avg_rating,similar_books
2509,6793408,NaN,NaN,A Study in Emerald,Neil Gaiman,2003.0,1.0,Alluding to both the Sherlock Holmes canon and...,"fantasy, paranormal, mystery, thriller, crime,...",https://s.gr-assets.com/assets/nophoto/book/11...,13290,337,2570,2025,819,156,54,5624,4.2,4593256
12299,43401121,NaN,NaN,Annie's Day,Andy Weir,2011.0,3.0,"I didn't want to wake up at 6am, especially on...","fiction, romance, fantasy, paranormal",https://images.gr-assets.com/books/1481823872m...,1882,455,254,412,283,80,41,1070,3.7,"44734637, 22025856, 15268747, 21883429, 185992..."
12122,22025856,NaN,NaN,Other People,Neil Gaiman,2001.0,3.0,"Free online fiction.\n""Time is fluid here,"" sa...","fantasy, paranormal, fiction, mystery, thrille...",https://images.gr-assets.com/books/1458233173m...,746,51,167,125,70,6,5,373,4.2,"43401121, 21883429, 16703904"


**NOTE TO SELF**:

* Goodreads says that 'A Study in Emerald' is one page. However, Google says 88 pages

In [23]:
# Check the percentage of missing data for each column
(books.isna().sum() / books.shape[0]).round(2)

work_id                      0.00
isbn                         0.15
isbn13                       0.12
original_title               0.00
author                       0.00
original_publication_year    0.00
num_pages                    0.05
description                  0.01
genres                       0.00
image_url                    0.00
reviews_count                0.00
text_reviews_count           0.00
5_star_ratings               0.00
4_star_ratings               0.00
3_star_ratings               0.00
2_star_ratings               0.00
1_star_ratings               0.00
ratings_count                0.00
avg_rating                   0.00
similar_books                0.19
dtype: float64

**NOTE TO SELF:**
* Figure out what to do with missing values later

**ALSO:**
* There are no text categorical columns

In [24]:
# Check for duplicates in the whole dataset
books.duplicated().sum()

np.int64(0)

In [25]:
# Check for duplicates book titles
books.duplicated(subset=['original_title', 'author']).sum()

np.int64(12)

In [26]:
books[books.duplicated(subset=['original_title', 'author'], keep=False)].sort_values('author')

,work_id,isbn,isbn13,original_title,author,original_publication_year,num_pages,description,genres,image_url,reviews_count,text_reviews_count,5_star_ratings,4_star_ratings,3_star_ratings,2_star_ratings,1_star_ratings,ratings_count,avg_rating,similar_books
12283,16736232,NaN,NaN,Pulled,Amy Lichtenhan,2011.0,242.0,There are some mistakes we make that we will r...,"romance, young-adult, fiction",https://images.gr-assets.com/books/1347389725m...,40489,1619,8416,8116,3825,1053,409,21819,4.1,"2421927, 21927322, 21767567, 18197231, 2186189..."
12289,15456915,NaN,NaN,Pulled,Amy Lichtenhan,2011.0,388.0,"For nine years, they drift through life, unabl...","romance, young-adult, fiction",https://images.gr-assets.com/books/1298429675m...,19447,879,3116,3121,1920,604,185,8946,3.9,"21986554, 15229756, 25068579, 21767567, 278706..."
2985,1460966,0689867042,9780689867040,Tithe: A Modern Faerie Tale,Holly Black,2002.0,332.0,Welcome to the realm of very scary faeries!\nS...,"fantasy, paranormal, young-adult, romance, fic...",https://images.gr-assets.com/books/1342120818m...,120875,3230,18977,18339,15306,5670,2826,61118,3.7,"3144132, 3514125, 2893, 990467"
4326,588218,0689868235,9780689868238,Tithe: A Modern Faerie Tale,Holly Black,2002.0,314.0,When seventeen-year-old Valerie runs away to N...,"fantasy, paranormal, young-adult, fiction, rom...",https://images.gr-assets.com/books/1332713354m...,46522,1327,9859,9218,6820,1901,546,28344,3.9,"2327820, 2227212, 2893, 990467"
1797,11163588,1606841750,9781606841754,Monsters,Ilsa J. Bick,2011.0,465.0,It could happen tomorrow . . .\nAn electromagn...,"young-adult, fiction, fantasy, paranormal, rom...",https://s.gr-assets.com/assets/nophoto/book/11...,55635,2799,7808,7838,4659,1322,620,22247,3.9,"14360639, 13996404, 14531613, 6914813, 1672861..."
9683,21699195,1455857408,9781455857401,Monsters,Ilsa J. Bick,2012.0,NaN,The Changed are on the move. The Spared are ou...,"young-adult, fiction, fantasy, paranormal, rom...",https://images.gr-assets.com/books/1372185588m...,10833,538,1319,1238,1011,416,206,4190,3.7,"18890761, 24160759, 21429037, 18365828"
931,385372,0061059927,9780061059926,Dark Reunion,L.J. Smith,1991.0,287.0,Elena rises from the dead to recreate the powe...,"young-adult, fantasy, paranormal, romance, fic...",https://s.gr-assets.com/assets/nophoto/book/11...,34047,678,5245,5285,6226,2684,1018,20458,3.5,3600162
3345,285261,1847386823,9781847386823,Dark Reunion,L.J. Smith,1991.0,732.0,GIFTED AND CURSED\nKaitlyn Fairchild has alway...,"young-adult, fantasy, paranormal, romance, fic...",https://images.gr-assets.com/books/1296605183m...,56464,1090,17441,9611,6107,1508,549,35216,4.2,"6674835, 7461960"
11042,7071165,0553269216,9780553269215,Rainbow Valley,L.M. Montgomery,1919.0,256.0,"Anne Shirley is grown up, has married her belo...","fiction, young-adult, history, historical fict...",https://images.gr-assets.com/books/1403192556m...,48160,790,10706,10556,7066,1398,221,29947,4.0,NaN
1423,1133797,0553213180,9780553213188,Rainbow Valley,L.M. Montgomery,1917.0,243.0,"Anne's own true love, Gilbert Blythe, is final...","fiction, young-adult, history, historical fict...",https://s.gr-assets.com/assets/nophoto/book/11...,89685,1481,26943,21263,11866,2270,959,63301,4.1,6582437


These are most likely updated versions of the same book. For now I will simply leave this.

I will need to clean the `genres` and `similar_books` columns by putting one value per column

In [27]:
# Turn each value in the genres column into a list
books['genres'] = books['genres'].str.split(', ')

# Create dummy columns for each genre and save it as a pd.DataFrame so it can be used as a lookup table
genres = (pd.get_dummies(
    books.explode('genres')[['work_id', 'genres']],
    columns=['genres'],
    prefix='',
    prefix_sep=''
               ).groupby('work_id')
                .sum()
                .reset_index()
)

# Inspect the genres table
genres.head()

,work_id,biography,children,comics,crime,fantasy,fiction,graphic,historical fiction,history,mystery,non-fiction,paranormal,poetry,romance,thriller,young-adult
0,104,1,0,0,0,0,1,0,1,1,0,1,0,0,1,0,0
1,114,1,0,0,0,1,1,0,1,1,0,0,1,0,1,0,0
2,115,1,1,0,0,0,1,0,1,1,0,0,0,0,0,0,1
3,423,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0
4,434,1,0,0,1,0,1,0,1,1,1,1,0,0,0,1,0


In [28]:
# Turn each value in the genres column into a list
books['similar_books'] = books['similar_books'].str.split(', ')

# Create dummy columns for each genre and save it as a pd.DataFrame so it can be used as a lookup table
similar_books = (pd.get_dummies(
    books.explode('similar_books')[['work_id', 'similar_books']],
    columns=['similar_books'],
    prefix='',
    prefix_sep=''
               ).groupby('work_id')
                .sum()
                .reset_index()
)

# Check the shape of similar books
similar_books.shape

(13525, 12116)

In [29]:
# Drop the genres and similar_books column from the original books table
books = books.drop(['genres', 'similar_books'], axis=1)

## Exporting

In [30]:
# Export the books, reviews, genres, and similar_books tables as pickle files
pd.to_pickle(books, '../Data/books.pkl')
pd.to_pickle(reviews, '../Data/reviews.pkl')
pd.to_pickle(genres, '../Data/genres.pkl')
pd.to_pickle(similar_books, '../Data/similar_books.pkl')